In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import pandas as pd
from time import sleep



In [68]:
class KakaoMapScraper:
    '''
    카카오맵 리뷰 데이터를 크롤링하는 클래스
    '''
    def __init__(self):
        # webdriver path, 카카오맵 url 설정
        self.driver = webdriver.Chrome(executable_path="./chromedriver")
        self.url = "https://map.kakao.com"
        self.driver.implicitly_wait(1)

        self.driver.get(self.url)
        self.driver.implicitly_wait(5)
    
    def _get_place_address_list(self, keywords):
        '''
        키워드 검색 결과의 장소들의 상세보기 주소를 스크래핑하여 반환
        param: keywords (type: list)
        return: place_address_list (type: dict)
        '''
        
        # 검색창 찾기
        search_area = self.driver.find_element(By.ID, "search.keyword.query") #.find_element_by_id("search.keyword.query")
        place_address_list = {}
        
        for keyword in keywords:
            # 키워드 검색
            search_area.clear()
            search_area.send_keys(keyword)
            self.driver.find_element(By.ID, "search.keyword.submit").send_keys(Keys.ENTER)
            self.driver.implicitly_wait(2)
            
            try: # 더보기 버튼이 있을시 클릭하여 진행
                self.driver.find_element(By.ID, "info.search.place.more").send_keys(Keys.ENTER)
                self.driver.implicitly_wait(2)
                
            except: # 1페이지만 존재할 때
                place_address_list[keyword]= []
                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                places = soup.select("ul.placelist > li.PlaceItem.clickArea > div.info_item > div.contact.clickArea")
                        
                # 현재 페이지 장소들의 상세보기 주소를 저장
                for place in places:
                    place_address_list[keyword].append(place.select_one("a")["href"])

                continue
                
            
            # 여러 페이지 존재할 경우, 1페이지부터 장소들의 상세보기 주소를 가져옴
            try:
                place_address_list[keyword]= []
                page_num = 1
                while True:
                    try:
                        # 페이지 이동
                        self.driver.find_element(By.ID, f"info.search.page.no{page_num}").send_keys(Keys.ENTER)#  find_element_by_id(f"info.search.page.no{page_num}").send_keys(Keys.ENTER)
                        sleep(1)

                        html = self.driver.page_source
                        soup = BeautifulSoup(html, "html.parser")
                        places = soup.select("ul.placelist > li.PlaceItem.clickArea > div.info_item > div.contact.clickArea")
                        
                        # 현재 페이지 장소들의 상세보기 주소를 저장
                        for place in places:
                            place_address_list[keyword].append(place.select_one("a")["href"])

                    except:
                        break
                    
                    page_num += 1
                    
                    # 5페이지가 넘어가면 다음 버튼을 클릭
                    if page_num == 6:
                        page_num = 1
                        try:
                            self.driver.find_element(By.ID,"info.search.page.next").send_keys(Keys.ENTER)
                            self.driver.implicitly_wait(2)
                        except:
                            break
            except:
                break

        return place_address_list
    
    def get_data(self, keywords):
        '''
        각 장소별 리뷰 데이터를 크롤링
        params: keywords (type: list)
        return: review_df (type: pd.DataFrame)
        '''
        
        # 키워드 장소들의 상세보기 주소를 가져옴
        place_address_list = self._get_place_address_list(keywords)
        
        # 리뷰 데이터를 저장할 데이터프레임 생성
        review_df = pd.DataFrame(columns=["name", "average_rating", "user", "user_rating", "comment", "url"])
        
        # 각 장소별 리뷰 데이터 크롤링
        for place, addresses in place_address_list.items():
            for address in addresses:
                # 각 장소의 상세보기 주소로 이동
                self.driver.get(address)
                sleep(2)

                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                
                # 장소 정보 저장(장소 이름, 분류, 평균 별점)
                place_info = soup.select_one("div.inner_place")
                #name = place_info.select_one("h2.tit_location").text
                #category = place_info.select_one("div.location_evaluation > span.txt_location").text[4:]
                average_rating = float(place_info.select_one("a.link_evaluation > span.color_b").text[:-1])
                location = self.driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]').text
                print(location)


                # 주소가 "서울 마포구"가 아닐 경우 크롤링을 진행하지 않음
                if location[:6] != "서울 마포구": continue
                
                # 1페이지의 후기부터 크롤링
                # page_num = 1
                # while True:
                #     # 후기 더보기 버튼 있을시 클릭
                #     buttons = self.driver.find_elements(By.CLASS_NAME, "btn_fold")#  .find_elements_by_class_name("btn_fold")
                #     for button in buttons:
                #         if button.is_displayed() and button.is_enabled():
                #             button.send_keys(Keys.ENTER)
                #     sleep(0.5)
                    
                #     # html 파싱
                #     html = self.driver.page_source
                #     soup = BeautifulSoup(html, "html.parser")
                #     reviews = soup.select("ul.list_evaluation > li")
                    
                #     # 각 후기별 정보 저장(작성자 이름, 작성자 별점, 작성자 후기)
                #     for review in reviews:
                #         user = review.select_one("a")["data-username"]
                #         rating_per = review.select_one("div.star_info > div > span > span")# > div.grade_star size_s > span.ico_star star_rate > span.ico_star inner_star")
                #         #print(rating_per)
                #         rating_per = rating_per["style"][6:]
                #         rating_per = rating_per[:-2]
                #         user_rating = float(rating_per)/20
                #         comment = review.select_one("div.comment_info > p.txt_comment > span").text
                        
                #         review_data = {"name": place, 
                #                        "average_rating": average_rating,
                #                        "user": user,
                #                        "user_rating": user_rating,
                #                        "comment": comment,
                #                        "url": address}
                #         review_df = review_df.append(review_data, ignore_index=True)
                    
                #     # 다음 후기 페이지가 있으면 이동
                #     try:
                #         page_num += 1
                #         self.driver.find_element_by_css_selector(f"a[data-page='{page_num}']").click()
                #         sleep(1)
                #     except: 
                #         break

                # 후기 크롤링
                while True:
                    # 후기 더보기 버튼 끝까지 클릭
                    button = self.driver.find_element(By.CLASS_NAME, 'link_more')
                    sleep(0.5)
                    btn_text = button.text
                    
                    if btn_text == "메뉴 더보기":
                        button = self.driver.find_elements(By.CLASS_NAME, 'link_more')
                        sleep(0.5)
                        btn_text = button[1].text
                        button = button[1]

                    if btn_text == "후기 접기":
                        break

                    button.click()
                
                # html 파싱
                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                reviews = soup.select("ul.list_evaluation > li")
                
                # 각 후기별 정보 저장(작성자 이름, 작성자 별점, 작성자 후기)
                for review in reviews:
                    user = review.select_one("a")["data-username"]
                    rating_per = review.select_one("div.star_info > div > span > span")# > div.grade_star size_s > span.ico_star star_rate > span.ico_star inner_star")
                    #print(rating_per)
                    rating_per = rating_per["style"][6:]
                    rating_per = rating_per[:-2]
                    user_rating = float(rating_per)/20
                    comment = review.select_one("div.comment_info > p.txt_comment > span").text
                    
                    review_data = {"name": place, 
                                    "average_rating": average_rating,
                                    "user": user,
                                    "user_rating": user_rating,
                                    "comment": comment,
                                    "url": address}
                    review_df = review_df.append(review_data, ignore_index=True)

        return review_df

In [69]:
scraper = KakaoMapScraper()

/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(executable_path="./chromedriver")


In [70]:
keywords = ["후계동", "카미야", "카레시"]
df = scraper.get_data(keywords)

서울 마포구 독막로 76 (우)04074
후기 더보기
후기 더보기
후기 접기


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed fro

서울 마포구 와우산로21길 28-6 지하1층 (우)04040
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 접기


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed fro

서울 마포구 독막로9길 31 (우)04048
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 더보기
후기 접기


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_17113/1789045714.py:208: FutureWarning: The frame.append method is deprecated and will be removed fro

In [71]:
df

,name,average_rating,user,user_rating,comment,url
0,후계동,4.5,ㅇㅇ,5.0,❤️❤️❤️❤️❤️,https://place.map.kakao.com/1183957472
1,후계동,4.5,핑구,5.0,오래 해주세요🙏,https://place.map.kakao.com/1183957472
2,후계동,4.5,:ᴅ,3.0,웨이팅해서 먹을 맛은 아니에요… 닭보쌈은 보통이었고 비빔국수라고 해서 당연히 국물 ...,https://place.map.kakao.com/1183957472
3,후계동,4.5,Vin,5.0,와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!,https://place.map.kakao.com/1183957472
4,후계동,4.5,손민영,5.0,오늘의 메뉴 너무 좋아요. 어쩜 갖가지 닭요리를 그렇게 잘하시죠?!! 이런 리뷰 잘...,https://place.map.kakao.com/1183957472
...,...,...,...,...,...,...
225,카레시,3.5,s,3.0,너무너무 오래기다렸어요 안에 테이블 비어있는데 왜 안부르는건지? 주방에 인력이 부족...,https://place.map.kakao.com/1570017396
226,카레시,3.5,Ellen,3.0,"웨이팅 앞쪽이었는데 밖에서 30분, 안에서 30분 기다렸고10개정도 테이블 중 5개...",https://place.map.kakao.com/1570017396
227,카레시,3.5,Youri Yamamoto,5.0,,https://place.map.kakao.com/1570017396
228,카레시,3.5,Mich,5.0,"정말 맛있었습니다. 한시간 넘게 밖에서 기다리고, 40분 정도 안에서 기다렸는데(음...",https://place.map.kakao.com/1570017396


In [45]:
df.shape

(9, 7)

In [ ]:
df.to_csv("헬스장_리뷰_데이터.csv")
df.to_excel("헬스장_리뷰_데이터.xlsx")